In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
np.random.seed(101)

# Number of samples
n_samples = 1000

# Generating data
x1 = np.random.uniform(0, 1, n_samples)  # Attribute 1
x2 = np.random.uniform(0, 1, n_samples)  # Attribute 2
x3 = np.random.uniform(0, 1, n_samples)  # Attribute 3

# Defining some nice effects for each class attribute
effect_x1 = np.sin(2 * np.pi * x1)  # Sinusoidal effect for attribute 1
effect_x2 = x2 ** 2  # Quadratic effect for attribute 2
effect_x3 = np.log(x3 + 1)  # Logarithmic effect for attribute 3

# Combining the effects with some weights and adding noise
y_effect = 1.5 * effect_x1 + 2 * effect_x2 - 1.2 * effect_x3
y_noise = np.random.normal(0, 0.1, n_samples)  # Adding some noise

# Redefining effects with slightly altered weights for multi-class distinction
y_effect_multi = 1.2 * effect_x1 + 2.5 * effect_x2 - 1.5 * effect_x3 + y_noise

# Creating three classes based on tertiles of the effect
percentiles = np.percentile(y_effect_multi, [33, 66])
y_class_multi = np.digitize(y_effect_multi, percentiles)

# Updating the DataFrame
data = pd.DataFrame({
    'Attribute1': x1,
    'Attribute2': x2,
    'Attribute3': x3,
    'Class': y_class_multi
})


data.head()


In [ ]:
from nampy.models import NAM

In [ ]:
nam = NAM(
    "Class ~ -1 + MLP(Attribute1; hidden_dims=[64, 32]) + MLP(Attribute2; hidden_dims=[64, 32]) + MLP(Attribute3)",   
    data=data, 
    feature_dropout=0.0001,
    classification=True,
    output_activation="sigmoid"
    )

In [ ]:
nam.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss={"output":"categorical_crossentropy"},
    metrics={"output":"accuracy"},
)

In [ ]:
# Train the model
nam.fit(nam.training_dataset, epochs=100, validation_data=nam.validation_dataset)

# Evaluate the model
loss = nam.evaluate(nam.validation_dataset)
print("Test Loss:", loss)

In [ ]:
nam.plot(interaction=False, interactive=False)